# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
!ls

CONTRIBUTING.md  data.sqlite  images  index.ipynb  LICENSE.md  README.md


In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

In [28]:
#Your code here; create the connection and cursor


SyntaxError: invalid syntax (<ipython-input-28-6a8994d586d6>, line 2)

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [6]:
#Your code here; use a subquery. No join will be necessary.
c.execute(""" select customerNumber,
                   contactLastName,
                   contactFirstName
                   from customers
                   join orders 
                   using(customerNumber)
                   where orderDate = '2003-01-31'; 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [7]:
c.execute(""" select customerNumber,
                   contactLastName,
                   contactFirstName
                   from customers
                  
                   where customerNumber in (SELECT customerNumber from orders 
                   where orderDate = '2003-01-31'); 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [10]:
#Your code here
c.execute(""" select *
                FROM orderDetails; 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


In [9]:
c.execute(""" select *
                FROM products; 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [16]:
c.execute(""" select productName,
                   sum(quantityOrdered)
                   from products
                   JOIN orderDetails
                   USING(productCode)
                    GROUP BY 1
                    ORDER BY sum(quantityOrdered) DESC
                      ;""")

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,sum(quantityOrdered)
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [24]:
c.execute(""" select *
                FROM orders; 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141


In [38]:
#Your code here

c.execute('''select productName, count(customerNumber) as num_custo
                FROM products
                
                JOIN orderdetails 
                USING(productCode)
                JOIN orders 
                USING(orderNumber)
                group by 1
                ORDER BY num_custo DESC
            ;''')
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
print('Number of results:', len(df))
df.head()

Number of results: 109


,productName,num_custo
0,1992 Ferrari 360 Spider red,53
1,18th Century Vintage Horse Carriage,28
2,1900s Vintage Bi-Plane,28
3,1900s Vintage Tri-Plane,28
4,1913 Ford Model T Speedster,28


In [36]:
c.execute("""select productName,
                    count(distinct customerNumber) numPurchasers
                    from products
                    join orderdetails
                    using(productCode)
                    join orders
                    using (orderNumber)
                    group by 1
                    order by numPurchasers desc
                    ;
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [39]:
c.execute(""" select *
                FROM employees; 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


In [41]:
#Your code here
c.execute("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         );
           """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1401,4,Paris,Pamela,Castillo
2,1621,5,Tokyo,Mami,Nishi
3,1501,7,London,Larry,Bott
4,1337,4,Paris,Loui,Bondur


## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [2]:
c.execute(""" select *
                FROM customers; 

                    """)

df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.00
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200.00
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700.00


In [16]:
#Your code here
c.execute("""select employeeNumber,
                    firstName,
                    lastName,
                    count(distinct customerNumber)
                    
                    FROM employees e
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    GROUP by 1
                    Having AVG(creditLimit) > 15000;
                                         """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,employeeNumber,firstName,lastName,count(distinct customerNumber)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!